In [3]:
import main
import model

### Reading Data

In [2]:
n_path = '../data/cora/graph.txt'
f_path = '../data/cora/feature.txt'
l_path = '../data/cora/group.txt'
graph, adj_mat, features = main.load_data(net_path=n_path, feature_path=f_path)

Loading data graph from ../data/cora/graph.txt
Loading data graph from ../data/cora/feature.txt
	Adjacency matrix shape: (2708, 2708)
	Feature matrix shape: (2708, 1433)
	Number of edges: 5429


### Configuring options

In [4]:
options = main.Options(number_of_nodes=adj_mat.shape[0], number_of_features=features.shape[1], rate=0.2)

### Training models

#### Node Model

In [5]:
node_model = model.NodeModel(options=options, adj=adj_mat, features=features)
node_model.train(epochs=10)
embeddings_nm = node_model.predict()
unf_embedding_nm = embeddings_nm[model.UNF_KEY]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/10
2708/2708 [==============================]2708/2708 [==============================] - 5s 2ms/step - loss: 0.6686 - unified_reconstruction_kernel_loss: 0.5214 - reconstruction_kernel_loss_1: 0.0745 - reconstruction_kernel_loss_2: 0.0727

Epoch 2/10
2708/2708 [==============================]2708/2708 [==============================] - 5s 2ms/step - loss: 0.4103 - unified_reconstruction_kernel_loss: 0.2710 - reconstruction_kernel_loss_1: 0.0739 - reconstruction_kernel_loss_2: 0.0654

Epoch 3/10
2708/2708 [==============================]2708/2708 [==============================] - 5s 2ms/step - loss: 0.3811 - unified_reconstruction_kernel_loss: 0.2528 - reconstruction_kernel_loss_1: 0.0680 - reconstruction_kernel_loss_2: 0.0603

Epoch 4/10
2708/2708 [==============================]2708/2708 [==============================] - 5s 2ms/step - loss: 0.3663 - unified_reconstruction_kernel_loss: 0.2462 - reconstr

#### Edge Model

In [6]:
edge_model = model.EdgeModel(options=options, adj=adj_mat, features=features, transferred_weights=node_model.get_learned_weights())
edge_model.train(epochs=1)
embeddings_em = edge_model.predict()
unf_embedding_em = embeddings_em[model.UNF_KEY]

Transferring learned weights to the 0-th layer of encoder
Weight shape: (2708, 512), bias shape: (512,)
Transferring learned weights to the 1-th layer of encoder
Weight shape: (512, 256), bias shape: (256,)
Transferring learned weights to the embedding layer
Transferring learned weights to the 0-th layer of decoder
Weight shape: (128, 256), bias shape: (256,)
Transferring learned weights to the 1-th layer of decoder
Weight shape: (256, 512), bias shape: (512,)
Transferring learned weights to reconstruction layer
Epoch 1/1
5429/5429 [==============================]5429/5429 [==============================] - 14s 3ms/step - loss: 0.8659 - reconstruction_kernel_loss_1: 0.0394 - reconstruction_kernel_loss_2: 0.0884 - unified_reconstruction_kernel_loss_1: 0.3446 - reconstruction_kernel_loss_3: 0.0772 - reconstruction_kernel_loss_4: 0.0485 - unified_reconstruction_kernel_loss_2: 0.2678



## Node Classification experiment

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

In [8]:
def node_classification(features, lbl_path, tr=.1, seed=0, cv=10):
    ss = model_selection.ShuffleSplit(n_splits=cv, train_size=tr, test_size=1. - tr, random_state=seed)
    label_df = pd.read_csv(lbl_path, header=None, sep='\t', names=['node', 'label'])
    labels = label_df.sort_values('node')['label'].values
    micros = []
    macros = []
    print('Training ratio: {}'.format(tr))
    clf = OneVsRestClassifier(LogisticRegression())
    for train_index, test_index in ss.split(features):
        x_train, x_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        pipe = make_pipeline(preprocessing.StandardScaler(), clf)
        y_hat = pipe.fit(x_train, y_train).predict(x_test)
        mic = f1_score(y_test, y_hat, average='micro')
        mac = f1_score(y_test, y_hat, average='macro')
        micros.append(mic)
        macros.append(mac)

    mic_mean, mic_std = np.mean(micros), np.std(micros)
    mac_mean, mac_std = np.mean(macros), np.std(macros)
    return mic_mean, mic_std, mac_mean, mac_std

#### Evaluation of the node model after 10 epoch

In [10]:
mic_f1, mic_std, mac_f1, mac_std  = node_classification(unf_embedding_nm, lbl_path=l_path)
mic_f1, mic_std, mac_f1, mac_std

Training ratio: 0.1


(0.5437243642329779,
 0.010746513535684997,
 0.516317399277235,
 0.01253198475756222)

#### Evaluation of the edge model after 1 epoch

In [11]:
mic_f1, mic_std, mac_f1, mac_std  = node_classification(unf_embedding_em, lbl_path=l_path)
mic_f1, mic_std, mac_f1, mac_std

Training ratio: 0.1


(0.52850697292863,
 0.0127183171513723,
 0.5004278587454828,
 0.021276662022361743)